# Workshop: Amazon ML Services for Customer Care (analyzing call center recordings)

You'll need an AWS Account, the [AWS CLI](https://aws.amazon.com/cli/), and NodeJS for this workshop. If this will be a challenge with using this note book, consider spinning up an Amazon EC2 instance. All the instructions for this lab are also available [here](https://s3.amazonaws.com/mast-mast-3/public/langws/2018/readme-cx.html)


        
## Overview

1. [Set up the prerequisites (AWS and NodeJS)](#setup)
1. [Amazon Comprehend](#comprehend)
1. [Amazon Comprehend for Sentiment Detection](#sentiment)
1. [Amazon Comprehend for Syntax Detection](#syntax)
1. [Optional: Amazon Translate](#translate)
1. [Optional: Amazon Translate with the CLI](#translatecli)
1. [Amazon Transcribe](#transcribe)
1. [Amazon Transcribe with the CLI](#transcribecli)
1. [Get source code](#sourcecode)
1. [Generate HTML Wrapper](#htmlwrapper)
1. [Deployment Considerations](#deployment)


Feel free to use your own device instead Jupyter Notebook

Too hard to get the prerequisites on your device? Consider using this Jupyter Notebook.

### Set up the prerequisites (AWS and NodeJS) <a class="anchor" id="setup"/>

Please use the credentials that were part of initial login screen. Set the env variables.

In [ ]:
import os
os.environ['AWS_DEFAULT_REGION'] = "us-east-1"
os.environ['AWS_ACCESS_KEY_ID'] = "<AWS_ACCESS_KEY_ID>"
os.environ['AWS_SECRET_ACCESS_KEY'] = "<AWS_SECRET_ACCESS_KEY>"
os.environ['AWS_SESSION_TOKEN'] = "<AWS_SESSION_TOKEN>"

Test previous aws configure is set properly by running following command

In [ ]:
!echo $AWS_SESSION_TOKEN

In [ ]:
# test it using something simple
!aws sagemaker list-notebook-instances

In [ ]:
# install jq
!sudo yum -y install jq

In [ ]:
# setup NodeJS, starting with the repository
!curl --silent --location https://rpm.nodesource.com/setup_10.x | sudo bash -

In [ ]:
!sudo yum clean all
!sudo yum -y install nodejs nodejs-devel nodejs-docs --enablerepo=nodesource

In [ ]:
# did it work?
!npm --version
!node --version

## Amazon Comprehend <a class="anchor" id="comprehend"/>
Let's check out the Amazon Comprehend console.

1. From the AWS Console, choose Amazon Comprehend (remember to choose a region like N. Virginia where the service is available)


2. Provide the following text and click Analyze

    ```
   With Easter around the corner and Venezuela facing a shortage of Communion wafers, a faith community across the border in Colombia stepped in with a donation of 250,000 of the holy discs. Venezuela's devastating food crisis means wheat flour has become a rare commodity in the country. Some churches have run out of the ingredient needed to make the sacramental bread that is central to celebrating the Holy Eucharist throughout the year -- and on Easter Sunday, according to local reports.
   A priest in the city of Mérida, in the Andes, told Catholic newspaper La Croix he and fellow ministers have had to "appeal to churchgoers to take flour to the nuns who make the hosts." So, members of the Catholic diocese of Cúcuta, Colombia, braved heavy rain this week to deliver the wafers over a bridge that connects the two countries "so that during this time of the Holy Week, (Venezuelans) can celebrate the central mysteries of the Christian faith," the community said in a statement on Facebook. Venezuela's economic crisis, fueled by a decline in oil production, shows no signs of improvement. People are starving because of routine food shortages. They are dying in hospitals because basic medicine and equipment aren't available.
    ```

    (original story from [CNN](https://www.cnn.com/2018/03/30/americas/colombia-venezuela-communion-wafers-easter-intl/index.html))


3. Which events do you see under `Entities`?


4. Examine the JSON response for one of these entities so you can see how `BeginOffset` could be used to highlight text.


5. What do you see under `Sentiment`?

## Amazon Comprehend for Sentiment Detection <a class="anchor" id="sentiment"/>

Let's try to use the [AWS CLI](https://aws.amazon.com/cli/) for sentiment detection.


1. Confirm you have the AWS CLI setup and configured using something like this `aws s3 ls`

In [ ]:
!aws s3 ls

2. Let's see how these one-liners are handled

In [ ]:
!aws comprehend detect-sentiment --region us-east-1 --language-code en --text 'You suck!'

In [ ]:
!aws comprehend detect-sentiment --region us-east-1 --language-code en --text 'This is the best thing since sliced bread.'

## Amazon Comprehend for Syntax Detection <a class="anchor" id="syntax"/>

Now let's try to identify some parts of speech using the command line.


In [ ]:
!aws comprehend detect-syntax --region us-east-1 --language-code en --text 'I love cloud!'

In [ ]:
!aws comprehend detect-syntax --region us-east-1 --language-code en --text 'The rain in Spain stays mainly in the plain.' 

In [ ]:
!aws comprehend detect-syntax --region us-east-1 --language-code en --text 'The rain in Spain stays mainly in the plain.' | jq -r '.SyntaxTokens[] | (.Text + " (" + .PartOfSpeech.Tag + ")")'

## Optional:  Amazon Translate <a class="anchor" id="translate"/>

Let's try out the Amazon Translate console.

1. From the AWS Console, choose Amazon Translate (remember to choose a region like N. Virginia where the service is available)
1. Leave the source language at `Auto (auto)` so the service can try to detect the language; set the target  language to `English (en)`
1. Paste the following text and click `Translate`
   ```
       Sempre fui uma repórter que adora uma festa de torcida. Não me importo com banho de cerveja, torcedor pulando, pisando no meu pé... sempre me deixo levar pela emoção e tento sentir o momento para fazer o meu trabalho da melhor maneira possível. Sempre me orgulhei por ter uma boa relação com todas as torcidas e por ser tratada com muito respeito!! Mas hoje, senti na pele a sensação de impotência que muitas mulheres sentem em estádios, metrôs, ou até mesmo andando pelasruas. Um beijo na boca, sem a minha permissão, enquanto eu exercia a minha profissão, que me deixou sem saber como agire sem entender como alguém pode se sentir no direito de agir assim. Com certeza o rapaz não sabe o quanto eu ralei paraestar ali. O quanto eu estudei e me esforcei para ter o prazer de poder contar histórias incríveis e estar em frente às câmeras mostrando tudo ao vivo. Faculdade, cursos, muitos finais de semana perdidos, muitos jogos de futebol analisados, estudo tático, técnico, pesquisas etc. Mas pelo simples fato de ser uma mulher no meio de uma torcida, nada disso tevevalor para ele. Se achou no direito de fazer o que fez. Hoje, me sinto ainda mais triste pelo que aconteceu comigo e pelo que acontece diariamente com muitas mulheres, mas sigo em frente como fiz ao vivo. Com a certeza que de cabeça erguida vamos conquistar o respeito que merecemos e que o cidadão que quis aparecer é quem deve se envergonhar do quefez. Sou repórter de futebol, sou mulher e mereço ser respeitada.
   ```
    (for more on this story about Bruna Dealtry, try [CNN](https://www.cnn.com/2018/03/30/americas/brazilian-sports-journalists-harassment-metoo-letherdoherjob/index.html))

## Optional:  Amazon Translate with the CLI <a class="anchor" id="translatecli"/>

Let's try to use the [AWS CLI](https://aws.amazon.com/cli/) to translate some text.

1. Confirm you have the AWS CLI setup and configured, using something like this

In [ ]:
!aws translate --region us-east-1 translate-text --source-language-code "en" --target-language-code "es" --text "Hello world"

2. It's often easier to provide paramaters as JSON; let's try that:

In [ ]:
!aws translate --region us-east-1 translate-text --cli-input-json \
'{ \
 "Text": "你好，世界", \
 "SourceLanguageCode": "zh", \
 "TargetLanguageCode": "en" \
}'

In [ ]:
!aws translate --region us-east-1 translate-text --cli-input-json \
'{ \
 "Text": "中央深改委首会，习近平如何谋划全面深改新阶段？", \
 "SourceLanguageCode": "zh", \
 "TargetLanguageCode": "en" \
}'

(for the original story, try [Xinhua News Agency](http://www.xinhuanet.com/politics/2018-03/30/c_1122613138.htm))

## Amazon Transcribe <a class="anchor" id="transcribe"/>

Let's try using the Amazon Transcribe Console to submit a transcription job.  (In the text below, substitute in values for variables like `_MY-BUCKET_`).

*Make sure your audio file is a two-channel 8 kHz (telephone quality) dialog. This simulates a caller-agent conversation. Generally, the turn detection and labeling program assumes the left channel is for the agent and the right for the caller. If that is not the case, we can change the code in `transcript-to-html.js`.*

1. Start by setting up an Amazon S3 location where you can place media (e.g., `s3://_MY-BUCKET_/_MY-PREFIX_`)
        
    - New to S3?  Try the [Amazon S3 - Getting Started Guide](https://docs.aws.amazon.com/AmazonS3/latest/gsg/CreatingABucket.html)
        
    - if bucket names are a challenge, consider using your initials and the date to generate a name

In [ ]:
import boto3
import json
import secrets
import string

with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    data = json.load(notebook_info)
    resource_arn = data['ResourceArn']
    region = resource_arn.split(':')[3]
print(region)

s3 = boto3.client('s3')
account_id = boto3.client('sts').get_caller_identity().get('Account')
bucket_name = account_id + "-comprehend-lab2-{}".format(''.join(
    secrets.choice(string.ascii_lowercase + string.digits) for i in range(8)))
print(bucket_name)

if region != "us-east-1":
    s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': region})
else:
    s3.create_bucket(Bucket=bucket_name)

2. Next, get a copy of the [example audio](https://s3.amazonaws.com/mast-mast-3/public/langws/2018/LDC97S42.wav) we will use
3. Upload the video to an Amazon S3 bucket so the service has access to it
     - *(the lab environment may have limited network bandwidth, so S3 cp might work best)*

In [ ]:
import os
os.environ['bucket_name'] = bucket_name

In [ ]:
!aws s3 cp s3://mast-mast-3/public/langws/2018/LDC97S42.wav s3://$bucket_name'/translate/'

4. Submit a transcription job using the console ([detailed instructions](https://docs.aws.amazon.com/transcribe/latest/dg/getting-started-asc-console.html))
        
    - Transcription job name:  `lab-01`
        
    - S3 input URL:  (S3 URI for media) (e.g., `s3://_MY-BUCKET_/_MY-PREFIX_/LDC97S42.wav`)
        
    - Language: `English`
        
    - Format:  `wav`
        
    - Channel identification: `Enabled`
        
    - accept default values for the rest and click `Create`
        
    

5. The transcription job should move to the `In progress` status and complete in a few minutes
6. When complete, download the results (JSON) and inspect the file; how would you find the duration of the transcribed audio?

## Amazon Transcribe with the CLI <a class="anchor" id="transcribecli"/>

Let's try to submit this job using the AWS CLI this time.

In [ ]:
# be sure to update the S3 URI before you submit this job
!aws s3 --region us-east-1 ls s3://$bucket_name/translate/
!aws transcribe --region us-east-1 start-transcription-job --cli-input-json \
'{\
    "TranscriptionJobName": "lab-02", \
    "LanguageCode": "en-US", \
    "MediaFormat": "wav", \
    "Media": { \
        "MediaFileUri": "s3://'$bucket_name'/translate/LDC97S42.wav" \
    }, \
    "Settings":  { \
        "ChannelIdentification": true \
    } \
}' 


In [ ]:
 # to view job status, consider something like this
!aws transcribe --region us-east-1 list-transcription-jobs

Ensure that above job is completed before executing next steps, outout of next step will be used later.

In [ ]:
!aws transcribe get-transcription-job --transcription-job-name lab-02 | jq -r '.TranscriptionJob.Transcript.TranscriptFileUri' 

## Get source code <a class="anchor" id="sourcecode"/>

We use a NodeJS script to generate formatted output files.  First, get the [source](https://s3.amazonaws.com/mast-mast-3/public/langws/2018/transcribe-utils-node.zip) and unzip the archive to a local directory.  
*Please ensure that the file downloaded from Amazon Transcribe is in the same directory as the unzipped source.*

In [ ]:
# be sure to correct the names/paths before using this
!mkdir transcribe-utils

In [ ]:
!cd transcribe-utils && wget https://s3.amazonaws.com/mast-mast-3/public/langws/2018/transcribe-utils-node.zip

In [ ]:
!cd transcribe-utils && unzip -o transcribe-utils-node.zip

In [ ]:
# fetch dependencies
!cd transcribe-utils && npm install

In [ ]:
# try it out (print usage instructions)
!cd transcribe-utils && node transcript-to-html.js

## Generate HTML Wrapper <a class="anchor" id="htmlwrapper"/>

We use a different NodeJS script to produce an HTML file for hosting (see **Generate Subtitles** above for source).

*The generated HTML refers to the audio file above, so please ensure it's in the same directory as the HTML file.*
*If you don't see subtitles or captions as an option for the video player, confirm that the browser able to download them.  You may need to use a local web server or host the files somewhere to overcome cross-origin challenges.*

Use the json link that we extracted abouve to download the output json file from lab-02 job.

In [ ]:
!cd transcribe-utils && wget "<signed ouput json file url >" -O lab-02.json

In [ ]:
!cd transcribe-utils && wget "https://s3.amazonaws.com/mast-mast-3/public/langws/2018/LDC97S42.wav" -O LDC97S42.wav

In [ ]:
!cd transcribe-utils && node transcript-to-html.js lab-02.json "LDC97S42.wav" "audio/wav" "en" > "LDC97S42.html"
# inspect the wrapper; try opening it with a web browser
# either use a local web server (express?) or an S3 bucket to host the files (HTML and WAV)

inspect the wrapper; try opening it with a web browser, the file LDC97S42.html that generated by transcript-to-html.js either use a local web server (express?) or an S3 bucket to host the files (HTML and WAV)

## Stretch Goals

- Use an Amazon CloudFront distribution to access files stored on Amazon S3
        
    - Take a look at the approach described [here](https://docs.aws.amazon.com/AmazonCloudFront/latest/DeveloperGuide/DownloadDistS3AndCustomOrigins.html#concept_S3Origin)
        
    - To publish files, consider something like this:  `aws s3 sync --exclude "*" --include "LDC97S42*" . s3://xxxlab20190408/public/`

- What would it take to provide a tag cloud of the key phrases for a transcript?
- How could you highlight turns in each conversation?

## Deployment Considerations <a class="anchor" id="deployment"/>

Imagine a service that did this automatically as audio files are uploaded.  How would you architect a solution that would automatically generate transcripts upon ingestion?


- What would the trigger be?
- How would you find the relevant service limits?
- What would you do with the output?  Does it need to be normalized?  Does it need to be available for searching?
- How would it scale (for context extraction and normalization)?
- Which security and cost considerations come to mind?
- Is there an AWS reference architecture you can start with?